# Basics of Differential Privacy

The following section briefly describes the basics of Differential Privacy (DP in short) and the motivation behind it. The interested reader can find more detailed information in the references at the end of the document. In particular, from the textbook of Cynthia Dwork [1] who is one of the main contributors to this new formalized concept.

Differential privacy seeks to postpone the inevitable fate of all privacy statements about a dataset. Namely, that after too many queries the privacy will be destroyed (this is known as the "Fundamental Law of Information Recovery"). With the advent of powerful machine learning applications, it is instrumental to provide guarantees that will secure the privacy of sensitive data. It is often the case that the trained models in machine learning learn more features than intended and with this possible sensible information of the dataset such as medical histories of patients or just the fact that someone is a members of a a particular dataset. 

One would naively think that "anonymization" of the dataset always works, but a careful look at it shows how weak this approach is: the richness of the data allows to "name" individuals even if the name is not included. Examples of this ocurred with the medical records of the governor of Massachussetts which were identified by matching different available datasets [3]; or the viewing histories of the subscribers present in the Netflix contest which were identified by linking such dataset to the public Internet Movie Database (IMDb) [4]. 

#### Promises of Differential Privacy

- Differential privacy is inmune to arbitrary post-processing attacks. A data analyst cannot lower the differential properties of the dataset. In particular, it is secure against linkage with past, present and future datasets and other sources of auxiliary information. 
- Differential privacy provides a quantitative measure of privacy loss. This allows to compare different techniques (such as PATE) given a target privacy guarantee. 
- Differential privacy behaves well under composition. This means that we can restrict our attention to the DP study of fundamental blocks.
- The definition of DP allows to take into consideration groups of people.

It is important to remark that DP **does not** promise to create privacy where there was not at the beginning. It just provides a rigorous guarantee that the same results of a differential-private dataset or learning algorithm are with high probability the same irrespective of the participation of a member (or group members). 

The following is the formal definition of DP due to Dwork [1]

**Definition (Differential Privacy)** A randomized mechanism $\mathcal{M}$ with domain $D$ and range $\mathcal{R}$ satisfies $(\varepsilon,\delta)-$differential privacy if for any two adjacent inputs $D,D'$ and for any subset of outputs $S\subset\mathcal{R}$ it holds that

$$\mathbb{P}[\mathcal{M}(D)\in S] \leq e^{\varepsilon}\ \mathbb{P}[\mathcal{M}(D')\in S]+\delta. $$

This definition of $(\varepsilon,\delta)-$differential privacy ensures, with probability $1-\delta$, that for all  adjacent datasets $D,D'$ 

$$\ln\left(\frac{\mathbb{P}[\mathcal{M}(D)\in S]}{\mathbb{P}[\mathcal{M}(D')\in S]}\right)\leq \varepsilon.$$

The quantity on the left hand side of the previous equation is known as *privacy loss*. From the definition of DP, we can also bound the expectation of an utility function over the database. In this context, a guarantee of $(\varepsilon,\delta)-$differential privacy of a dabaset just promises that the expected utility of participating will be at most harmed by a factor of $\exp(\varepsilon)\approx (1+\varepsilon)$.

The random mechanism that we will interested here is the *Private Aggregation of Teacher Ensembles* or just *PATE* [2]. We will describe it in the next section.

# Private Aggregation of Teacher Ensembles (PATE)

Let us describe the general setup. We consider a data holder, or *curator*, that has access to a private dataset and wants to provide a learning algorithm which captures the main features of the data and such that is differentiable private. For that matters, the curator trains an ensemble of "teacher" models with the private data and then transfers this knowledge to a single "student" model. The "student" model is the final product that captures the main features and information of the private dataset. It is the learning algorithm that the curator wants to make public. However, in order to avoid adversary attacks that could lead to distinguish individual teacher's private knowledge, we add some random noise. This random noise is directly linked with the privacy content that the curator wishes to have on the dataset. 

The training of the student follows an aggregation model which we describe now. Let $m$ be the number of classes in which the private data falls. Let $\{h_{i}\}, i=1,...,N$ be the teachers ensemble where $N$ is the total number of teachers. Given an input $x\in D$, let 

$$n_{j}(x)=|\{i: i\in\{1,..,N\}, h_{i}(x)=j\}|$$ 

be the number of teachers that assigned the class $j$ to $x$. Then the aggregated knowledge used for training the student is

$$h(x)=\arg \max_{j}\{n_{j}(x)+ Y\}.$$

Here $Y$ is random variable representing the noise. The function $h$ is used train the student, by relabeling a part of the private dataset. We use here the *Laplace mechanism* which considers as noise $Y$ a random variable distributed according to the Laplace distribution

$$Lap(x|b)=\frac{1}{2b}\ \exp\left[{-\frac{|x|}{b}}\right].$$

It turns out that when we consider $Lap(\cdot|\tfrac{1}{\varepsilon})$ the Laplace mechanism is $(\varepsilon,0)-$differential private as proven in Theorem 3.6 of Ref. [1]. The PATE model is summarized in Figure 1 below.

We will implement PATE in the MNIST und Fashion MNIST dataset. Both of this datasets consist of 10 classes and a training set of 60,000 examples, and a test set of 10,000 examples. The MNIST consist of hand-written digits from 0 to 9. On the other hand, Fashion-MNIST is a dataset of [Zalando's article images](https://github.com/zalandoresearch/fashion-mnist).  One could interpret a sucessful DP-algorithm in the Fashion-MNIST as an algorithm that describes with high precision the 10 classes of articles and such that it protects the privacy of the individual brands that are involved in the dataset. We will be using Pytorch for the implementation of the PATE model.


![PATE_Setup](pate "PATE Setup")

### References

1. C. Dwork and A. Roth. *The algorithmic Foundations of Differential Privacy*, Now Publishers Inc (2014)

2. N. Papernot, M. Abadi, U. Erlingsson, I. Goodfellow and K. Talwar. *Semi-supervised Knowledge Transfer for Deep Learning From Private Training Data*.
- L. Sweeney. *Weaving Technology and Policy Together to Maintain Confidentiality*, Journal of Law, Medicines Ethics, 25:98-110, 1997.
- A. Narayan and V. Shmatikov. *Robust de-anonymization of large sparse datasets (how to break anonymity of the netflix prize dataset)*. In Proceedings of IEEE Symposium on Security and Privacy 2008.